In [1]:
import mido

In [2]:
mid = mido.MidiFile('./Tetris.mid')

In [3]:
for i, track in enumerate(mid.tracks):
    print(i, track.name)
print(mid.ticks_per_beat)

0 
1 Acoustic Grand Piano
2 Instrument2
1024


In [4]:
for msg in mid.tracks[1]:
    print(msg)

<meta message device_name name='SmartMusic SoftSynth 1' time=0>
<meta message track_name name='Acoustic Grand Piano' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=101 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=7 value=110 time=0
control_change channel=0 control=100 value=0 time=3
control_change channel=0 control=101 value=0 time=0
control_change channel=0 control=6 value=12 time=1
control_change channel=0 control=38 value=0 time=1
note_on channel=0 note=71 velocity=61 time=5
note_on channel=0 note=76 velocity=75 time=0
note_off channel=0 note=71 velocity=0 time=1043
note_off channel=0 note=76 velocity=0 time=0
note_on channel=0 note=68 velocity=57 time=1
note_on channel=0 note=71 velocity=71 time=0
note_off channel=0 note=68 velocity=0 time=474
note_off channel=0 note=71 velocity=0 time=30
note_on channel=0 note=69 velocity=56 time=1
note_on channel=0 note=72 velocity=70 time=0
note_off channe

In [5]:
piano_track = mid.tracks[1]

In [6]:
mid.ticks_per_beat

1024

In [7]:
from collections import namedtuple

MIDINote = namedtuple('MIDINote', ['note', 'volume'])
MIDIState = namedtuple('MIDIState', ['time','notes','tempo'])

In [8]:
from copy import deepcopy

In [9]:
state_array = []
state = MIDIState(0, set(), 0)

trk_indexes = [0] * len(mid.tracks)
trk_offset_times = [0] * len(mid.tracks)
while True:
    prev_state = deepcopy(state)
    # First, consume all messages up to the current time
    for i, trk in enumerate(mid.tracks):
        cmd_cnt = 0
        while trk_indexes[i] < len(trk):
            msg = trk[trk_indexes[i]]
            if msg.time + trk_offset_times[i] <= state.time:
                trk_offset_times[i] += msg.time
                trk_indexes[i] += 1
                if msg.type == 'set_tempo': 
                    cmd_cnt += 1
                    state = MIDIState(state.time, state.notes, msg.tempo)
                elif msg.type == 'note_on':
                    cmd_cnt += 1
                    state = MIDIState(state.time, state.notes.union({MIDINote(msg.note, int(msg.velocity/10))}), state.tempo)
                elif msg.type == 'note_off':
                    cmd_cnt += 1
                    state = MIDIState(state.time, set(filter(lambda n: n.note != msg.note, state.notes)), state.tempo)
            else:
                break
        #if cmd_cnt > 0:
            #print('Executed %d commands at time %d for track %d' % (cmd_cnt, state.time, i))
    if prev_state != state:
        #print(state)
        #print('Logging state %s at time %d' % (state, state.time))
        state_array.append(state)
        
    # Locate next command to execute
    next_time = 0
    for i, trk in enumerate(mid.tracks):
        if trk_indexes[i] < len(trk):
            next_msg_time = trk[trk_indexes[i]].time - (state.time - trk_offset_times[i])
            if next_time == 0 or next_msg_time < next_time: next_time = next_msg_time
    if next_time == 0:
        break
    #print('Advancing by %d to time %d' % (next_time, state.time + next_time))
    state = MIDIState(state.time + next_time, state.notes, state.tempo)
    
for i,s in enumerate(state_array):
    sorted_notes = list(sorted(s.notes, key=lambda n: n.note))
    sorted_notes = sorted_notes[-2:]
    state_array[i] = MIDIState(s.time, sorted_notes, s.tempo)
    print(state_array[i])

MIDIState(time=0, notes=[], tempo=500003)
MIDIState(time=10, notes=[MIDINote(note=71, volume=6), MIDINote(note=76, volume=7)], tempo=500003)
MIDIState(time=546, notes=[MIDINote(note=71, volume=6), MIDINote(note=76, volume=7)], tempo=500003)
MIDIState(time=576, notes=[MIDINote(note=71, volume=6), MIDINote(note=76, volume=7)], tempo=500003)
MIDIState(time=1053, notes=[MIDINote(note=52, volume=5)], tempo=500003)
MIDIState(time=1054, notes=[MIDINote(note=68, volume=5), MIDINote(note=71, volume=7)], tempo=500003)
MIDIState(time=1084, notes=[MIDINote(note=68, volume=5), MIDINote(note=71, volume=7)], tempo=500003)
MIDIState(time=1528, notes=[MIDINote(note=40, volume=5), MIDINote(note=71, volume=7)], tempo=500003)
MIDIState(time=1558, notes=[MIDINote(note=40, volume=5)], tempo=500003)
MIDIState(time=1559, notes=[MIDINote(note=69, volume=5), MIDINote(note=72, volume=7)], tempo=500003)
MIDIState(time=1589, notes=[MIDINote(note=69, volume=5), MIDINote(note=72, volume=7)], tempo=500003)
MIDIState(

In [10]:
def byte_enc(byte):
    return '{0:#04x}'.format(byte)

def get_byte_str(bytes):
    return ','.join(map(byte_enc, bytes))

def varint_encode(value):
    bytes = [value & 0x7f]
    while (value > 0):
        value = value >> 7
        if (value > 0):
            bytes.append(value & 0x7f)
    bytes.reverse()
    bytes[-1] = bytes[-1] + 0x80
    return bytes

def encode_midi_note_on(time, note, volume, timer1):
    bytes = varint_encode(time)
    bytes.append((0x80 if timer1 else 0) + note)
    bytes.append(volume)
    print('%30s, //%6d %s ON, NOTE %d VOLUME %d' %
          (get_byte_str(bytes), time, 'T1' if timer1 else 'T2', note, volume))
    return bytes

def encode_midi_note_off(time, timer1):
    bytes = varint_encode(time)
    bytes.append(0x80 if timer1 else 0x00)
    print('%30s, //%6d %s OFF' %
          (get_byte_str(bytes), time, 'T1' if timer1 else 'T2'))
    return bytes    

def encode_midi_both_off(time):
    bytes = varint_encode(time)
    bytes.append(0x01)
    print('%30s, //%6d BOTH OFF' %
          (get_byte_str(bytes), time))
    return bytes        

def encode_midi_change_tempo(time,tempo):
    bytes = varint_encode(time)
    bytes.append(0x02)
    bytes += varint_encode(tempo)
    print('%30s, //%6d TEMPO > %d' %
          (get_byte_str(bytes), time,tempo))
    return bytes        
    
def encode_midi_end_program(time):
    bytes = varint_encode(time)
    bytes.append(0x04)
    print('%30s  //%6d END OF FILE' %
          (get_byte_str(bytes), time))
    return bytes        

In [11]:
encode_midi_note_on(1056,56,5,False)
encode_midi_note_off(64,True)
encode_midi_both_off(12000)
encode_midi_change_tempo(152,141000)
encode_midi_end_program(34)

           0x08,0xa0,0x38,0x05, //  1056 T2 ON, NOTE 56 VOLUME 5
                     0xc0,0x80, //    64 T1 OFF
                0x5d,0xe0,0x01, // 12000 BOTH OFF
 0x01,0x98,0x02,0x08,0x4d,0xc8, //   152 TEMPO > 141000
                     0xa2,0x04  //    34 END OF FILE


[162, 4]

In [12]:
current_tempo = state_array[0].tempo
midi_data = encode_midi_change_tempo(0, current_tempo)
for i,s in enumerate(state_array):
    if i == 0:
        prev_state = MIDIState(time=0, notes=set(), tempo=current_tempo)
    else:
        prev_state = state_array[i - 1]
    time_diff = s.time - prev_state.time
    
    if s.tempo != prev_state.tempo:
        midi_data += encode_midi_change_tempo(time_diff, s.tempo)
        time_diff = 0
    
    timers_off = [False] * 2
    for timer_num in range(2):
        if len(s.notes) <= timer_num:
            if len(prev_state.notes) > timer_num:
                timers_off[timer_num] = True
        else:
            curr_note = s.notes[timer_num]
            if len(prev_state.notes) <= timer_num or prev_state.notes[timer_num] != curr_note:
                midi_data += encode_midi_note_on(time_diff,curr_note.note,curr_note.volume,timer_num==1)
                time_diff = 0
    if timers_off[0] and timers_off[1]:
        midi_data += encode_midi_both_off(time_diff)
        time_diff = 0
    elif timers_off[0]:
        midi_data += encode_midi_note_off(time_diff,False)
        time_diff = 0
    elif timers_off[1]:
        midi_data += encode_midi_note_off(time_diff,True)
        time_diff = 0
midi_data += encode_midi_end_program(0)

      0x80,0x02,0x1e,0x42,0xa3, //     0 TEMPO > 500003
                0x8a,0x47,0x06, //    10 T2 ON, NOTE 71 VOLUME 6
                0x80,0xcc,0x07, //     0 T1 ON, NOTE 76 VOLUME 7
           0x03,0xdd,0x34,0x05, //   477 T2 ON, NOTE 52 VOLUME 5
                     0x80,0x80, //     0 T1 OFF
                0x81,0x44,0x05, //     1 T2 ON, NOTE 68 VOLUME 5
                0x80,0xc7,0x07, //     0 T1 ON, NOTE 71 VOLUME 7
           0x03,0xbc,0x28,0x05, //   444 T2 ON, NOTE 40 VOLUME 5
                     0x9e,0x80, //    30 T1 OFF
                0x81,0x45,0x05, //     1 T2 ON, NOTE 69 VOLUME 5
                0x80,0xc8,0x07, //     0 T1 ON, NOTE 72 VOLUME 7
           0x03,0xc1,0x34,0x05, //   449 T2 ON, NOTE 52 VOLUME 5
                     0x9e,0x80, //    30 T1 OFF
                0x81,0x47,0x05, //     1 T2 ON, NOTE 71 VOLUME 5
                0x80,0xca,0x07, //     0 T1 ON, NOTE 74 VOLUME 7
 0x01,0xcd,0x02,0x1e,0x64,0xf2, //   205 TEMPO > 504434
           0x01,0xf4,0x28,0x0

In [13]:
get_byte_str(midi_data)

'0x80,0x02,0x1e,0x42,0xa3,0x8a,0x47,0x06,0x80,0xcc,0x07,0x03,0xdd,0x34,0x05,0x80,0x80,0x81,0x44,0x05,0x80,0xc7,0x07,0x03,0xbc,0x28,0x05,0x9e,0x80,0x81,0x45,0x05,0x80,0xc8,0x07,0x03,0xc1,0x34,0x05,0x9e,0x80,0x81,0x47,0x05,0x80,0xca,0x07,0x01,0xcd,0x02,0x1e,0x64,0xf2,0x01,0xf4,0x28,0x05,0x8c,0x02,0x1e,0x74,0xc0,0x92,0x80,0x81,0x34,0x05,0x80,0xcc,0x06,0x01,0xca,0x80,0x85,0x02,0x1f,0x06,0x99,0x8b,0xca,0x06,0x01,0xf1,0x80,0x84,0x02,0x1f,0x28,0x85,0x8c,0x45,0x05,0x80,0xc8,0x07,0x01,0xd6,0x02,0x1f,0x38,0x95,0x01,0xe6,0x28,0x05,0x9a,0x02,0x1f,0x4a,0xbc,0x84,0x80,0x81,0x44,0x05,0x80,0xc7,0x07,0x01,0xdd,0x02,0x1f,0x5a,0xef,0x01,0xe1,0x34,0x05,0x9e,0x80,0x85,0x40,0x06,0x80,0xc5,0x07,0x01,0xe2,0x02,0x1e,0x42,0xa3,0x03,0xdd,0x39,0x05,0x80,0x80,0x81,0x40,0x05,0x80,0xc5,0x07,0x03,0xbc,0x2d,0x05,0x9e,0x80,0x81,0x45,0x05,0x80,0xc8,0x07,0x03,0xc1,0x39,0x05,0x9e,0x80,0x81,0x48,0x05,0x80,0xcc,0x07,0x03,0xda,0x39,0x05,0x80,0x80,0x81,0x47,0x05,0x80,0xca,0x07,0x03,0xbc,0x2d,0x05,0x9e,0x80,0x81,0x45,0x05,0x80

In [14]:
for msg in mid.tracks[1]:
    print(msg)

<meta message device_name name='SmartMusic SoftSynth 1' time=0>
<meta message track_name name='Acoustic Grand Piano' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=101 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=7 value=110 time=0
control_change channel=0 control=100 value=0 time=3
control_change channel=0 control=101 value=0 time=0
control_change channel=0 control=6 value=12 time=1
control_change channel=0 control=38 value=0 time=1
note_on channel=0 note=71 velocity=61 time=5
note_on channel=0 note=76 velocity=75 time=0
note_off channel=0 note=71 velocity=0 time=1043
note_off channel=0 note=76 velocity=0 time=0
note_on channel=0 note=68 velocity=57 time=1
note_on channel=0 note=71 velocity=71 time=0
note_off channel=0 note=68 velocity=0 time=474
note_off channel=0 note=71 velocity=0 time=30
note_on channel=0 note=69 velocity=56 time=1
note_on channel=0 note=72 velocity=70 time=0
note_off channe

In [15]:
MIDI_TICKS_PER_BEAT = mid.ticks_per_beat
ts_msg = next(m for m in mid if m.type == 'time_signature')
MIDI_TS_NUM = ts_msg.numerator
MIDI_TS_DEN = ts_msg.denominator

In [16]:
OUTPUT_TICKS_PER_BAR = 1024
MIDI_TICKS_PER_OUTPUT_TICK = (MIDI_TICKS_PER_BEAT * 4) / OUTPUT_TICKS_PER_BAR
state_array_condensed = deepcopy(state_array)

# First, remove messages that occur within the same output tick
i = 1
prev_tick = int(state_array_condensed[0].time / MIDI_TICKS_PER_OUTPUT_TICK)
while i < len(state_array_condensed):
    output_tick = int(state_array_condensed[i].time / MIDI_TICKS_PER_OUTPUT_TICK)
    if output_tick <= prev_tick:
        state_array_condensed.pop(i - 1)
    else:
        prev_tick = output_tick
        i += 1
        
# Now remove all but the highest two notes
def drop_low_notes(s):
    hi_notes = sorted(s.notes, key=lambda n:n.note)[-2:]
    return MIDIState(s.time, hi_notes, s.tempo)
state_array_condensed = list(map(drop_low_notes, state_array_condensed))

for i, s in enumerate(state_array_condensed):
    if i > 0:
        prev_s = state_array_condensed[i - 1]
    else:
        prev_s = MIDIState(0, [MIDINote(0,0), MIDINote(0,0)], 0)
    note_num = len(s.notes)
    new_notes = s.notes + prev_s.notes[note_num:]
    new_notes = list(map(lambda n:MIDINote(n.note, int(n.volume/10)), new_notes))
    state_array_condensed[i] = MIDIState(s.time, new_notes, s.tempo)

In [17]:
MIDI_TICKS_PER_OUTPUT_TICK

4.0

In [18]:
sorted(state_array[1].notes, key=lambda n:n.note)[-2:]

[MIDINote(note=71, volume=6), MIDINote(note=76, volume=7)]

In [19]:
state_array_condensed

[MIDIState(time=0, notes=[MIDINote(note=0, volume=0), MIDINote(note=0, volume=0)], tempo=500003),
 MIDIState(time=10, notes=[MIDINote(note=71, volume=0), MIDINote(note=76, volume=0)], tempo=500003),
 MIDIState(time=546, notes=[MIDINote(note=71, volume=0), MIDINote(note=76, volume=0)], tempo=500003),
 MIDIState(time=576, notes=[MIDINote(note=71, volume=0), MIDINote(note=76, volume=0)], tempo=500003),
 MIDIState(time=1054, notes=[MIDINote(note=68, volume=0), MIDINote(note=71, volume=0)], tempo=500003),
 MIDIState(time=1084, notes=[MIDINote(note=68, volume=0), MIDINote(note=71, volume=0)], tempo=500003),
 MIDIState(time=1528, notes=[MIDINote(note=40, volume=0), MIDINote(note=71, volume=0)], tempo=500003),
 MIDIState(time=1559, notes=[MIDINote(note=69, volume=0), MIDINote(note=72, volume=0)], tempo=500003),
 MIDIState(time=1589, notes=[MIDINote(note=69, volume=0), MIDINote(note=72, volume=0)], tempo=500003),
 MIDIState(time=2038, notes=[MIDINote(note=52, volume=0), MIDINote(note=72, volume

In [20]:
def byte_enc(byte):
    return '{0:#04x}'.format(byte)

def hex_encode(value, bytes):
    def grab():
        nonlocal value
        byte = value & 0xff
        value = value >> 8
        return byte_enc(byte)
    bytestr = [grab() for b in range(bytes)]
    #bytestr.reverse()
    return bytestr

def make_ts(tone = [None, None], volume = [None, None],
            end_program = False, timer1_equals_timer2 = False, new_tempo = None,
            num_led_inst = 0, beat_number = 0):
    def opcode(idx):
        return (1 if tone[idx] is not None else 0) + (2 if volume[idx] is not None else 0)
    ts = hex_encode((beat_number & 0xffff) \
                + ((num_led_inst & 0x1f) << 16) \
                + (1 << 21 if new_tempo else 0) \
                + (1 << 22 if timer1_equals_timer2 else 0) \
                + (1 << 23 if end_program else 0) \
                + (opcode(1) << 24) \
                + (opcode(0) << 28), 4)
    if end_program: return ts
    if tone[0] is not None: ts.append(byte_enc(tone[0]))
    if volume[0] is not None: ts.append(byte_enc(volume[0]))
    if not timer1_equals_timer2:
        if tone[1] is not None: ts.append(byte_enc(tone[1]))
        if volume[1] is not None: ts.append(byte_enc(volume[1]))
    if new_tempo:
        ts += hex_encode(new_tempo, 4)
    return ts

In [21]:
make_ts(tone=[132,None])

['0x00', '0x00', '0x00', '0x10', '0x84']

In [22]:
MAX_MIDI_BAR = 64
MAX_MIDI_TIME = MAX_MIDI_BAR * 4 * MIDI_TICKS_PER_BEAT
buffer = []
for i, s in enumerate(state_array_condensed):
    if s.time >= MAX_MIDI_TIME:
        break
    beat_num = int(s.time / MIDI_TICKS_PER_OUTPUT_TICK)
    if i == 0:
        prev_s = MIDIState(0, [], 0)
    else:
        prev_s = state_array_condensed[i - 1]
    tone = []
    volume = []
    msg = []
    req_ts = False
    for timer_id in range(2):
        if len(s.notes) > timer_id:
            if len(prev_s.notes) <= timer_id or prev_s.notes[timer_id].note != s.notes[timer_id].note:
                tone.append(s.notes[timer_id].note)
                msg.append('T%d NOTE->%d' % (timer_id + 1, s.notes[timer_id].note))
                req_ts = True
            else:
                tone.append(None)
            if len(prev_s.notes) <= timer_id or prev_s.notes[timer_id].volume != s.notes[timer_id].volume:
                volume.append(s.notes[timer_id].volume)                
                msg.append('T%d VOL->%d' % (timer_id + 1, s.notes[timer_id].volume))
                req_ts = True
            else:
                volume.append(None)
        else:
            msg.append('T%d OFF' % (timer_id + 1))
            req_ts = True
            tone.append(None)
            if prev_s.volume > 0: volume.append(0)
    new_tempo = s.tempo if s.tempo != prev_s.tempo else None
    if new_tempo:
        msg.append('TEMPO->%d' % new_tempo)
        req_ts = True
    if req_ts:
        ts = make_ts(tone=tone, volume=volume, beat_number=beat_num, new_tempo=new_tempo)
        buffer += ts
        print('%8d - %s\n\t\t%s' % (beat_num, ts, ', '.join(msg)))
buffer += make_ts(end_program=True)
print(len(buffer))

       0 - ['0x00', '0x00', '0x20', '0x33', '0x00', '0x00', '0x00', '0x00', '0x23', '0xa1', '0x07', '0x00']
		T1 NOTE->0, T1 VOL->0, T2 NOTE->0, T2 VOL->0, TEMPO->500003
       2 - ['0x02', '0x00', '0x00', '0x11', '0x47', '0x4c']
		T1 NOTE->71, T2 NOTE->76
     263 - ['0x07', '0x01', '0x00', '0x11', '0x44', '0x47']
		T1 NOTE->68, T2 NOTE->71
     382 - ['0x7e', '0x01', '0x00', '0x10', '0x28']
		T1 NOTE->40
     389 - ['0x85', '0x01', '0x00', '0x11', '0x45', '0x48']
		T1 NOTE->69, T2 NOTE->72
     509 - ['0xfd', '0x01', '0x00', '0x10', '0x34']
		T1 NOTE->52
     517 - ['0x05', '0x02', '0x00', '0x11', '0x47', '0x4a']
		T1 NOTE->71, T2 NOTE->74
     576 - ['0x40', '0x02', '0x20', '0x00', '0x72', '0xb2', '0x07', '0x00']
		TEMPO->504434
     637 - ['0x7d', '0x02', '0x00', '0x10', '0x28']
		T1 NOTE->40
     640 - ['0x80', '0x02', '0x20', '0x00', '0x40', '0xba', '0x07', '0x00']
		TEMPO->506432
     644 - ['0x84', '0x02', '0x00', '0x11', '0x34', '0x4c']
		T1 NOTE->52, T2 NOTE->76
     704 - ['

In [23]:
len(buffer)

945

In [24]:
buffer

['0x00',
 '0x00',
 '0x20',
 '0x33',
 '0x00',
 '0x00',
 '0x00',
 '0x00',
 '0x23',
 '0xa1',
 '0x07',
 '0x00',
 '0x02',
 '0x00',
 '0x00',
 '0x11',
 '0x47',
 '0x4c',
 '0x07',
 '0x01',
 '0x00',
 '0x11',
 '0x44',
 '0x47',
 '0x7e',
 '0x01',
 '0x00',
 '0x10',
 '0x28',
 '0x85',
 '0x01',
 '0x00',
 '0x11',
 '0x45',
 '0x48',
 '0xfd',
 '0x01',
 '0x00',
 '0x10',
 '0x34',
 '0x05',
 '0x02',
 '0x00',
 '0x11',
 '0x47',
 '0x4a',
 '0x40',
 '0x02',
 '0x20',
 '0x00',
 '0x72',
 '0xb2',
 '0x07',
 '0x00',
 '0x7d',
 '0x02',
 '0x00',
 '0x10',
 '0x28',
 '0x80',
 '0x02',
 '0x20',
 '0x00',
 '0x40',
 '0xba',
 '0x07',
 '0x00',
 '0x84',
 '0x02',
 '0x00',
 '0x11',
 '0x34',
 '0x4c',
 '0xc0',
 '0x02',
 '0x20',
 '0x00',
 '0x19',
 '0xc3',
 '0x07',
 '0x00',
 '0xc2',
 '0x02',
 '0x00',
 '0x01',
 '0x4a',
 '0x00',
 '0x03',
 '0x20',
 '0x00',
 '0x05',
 '0xd4',
 '0x07',
 '0x00',
 '0x03',
 '0x03',
 '0x00',
 '0x11',
 '0x45',
 '0x48',
 '0x40',
 '0x03',
 '0x20',
 '0x00',
 '0x15',
 '0xdc',
 '0x07',
 '0x00',
 '0x79',
 '0x03',
 '0x00',
 